# CR Project : Nanogram solver

## Présentation du Projet
Ce projet explore l’application des solveurs SAT et des problèmes de satisfaction de contraintes (CSP) pour résoudre des puzzles logiques, en particulier les **nonogrammes** (également appelés Picross ou Hanjie). L’objectif est de développer un **solveur automatisé** capable de déterminer si un nonogramme donné (2D et 3D) a une solution et, le cas échéant, de trouver cette solution efficacement.

Nous procéderons en deux phases :
1. **Résolution des nonogrammes 2D à l’aide d’un solveur SAT**
2. **Extension de la solution aux nonogrammes 3D**, en tenant compte de l’explosion combinatoire et en justifiant une approche de résolution ligne par ligne.

Les nonogrammes ont gagné en popularité grâce à divers jeux, notamment la série *Nintendo Picross 3D*, sortie pour la première fois en 2009, qui a introduit le concept de nanogrammes en trois dimensions.

---



## Partie 1 : Solveur de Nonogrammes 2D

### Définition du Problème
Un **nonogramme 2D** est constitué d’une grille où chaque ligne et chaque colonne est accompagnée d’une série de nombres. Ces nombres indiquent les blocs de cellules remplies consécutives, séparées par au moins une cellule vide.

#### Exemple d’entrée (cactus.txt) :
```
10  # Nombre de lignes
8   # Nombre de colonnes
2
2 2
2 2
2 2 2 # Exemple pour cette ligne on sait qu'il 3 blocs de 2 cases consécutives séparées par un nombre inconnu de cases vides
2 2 2
2 5
2 2
5
2
2
5
5
1
10
10
1
5
5
```

#### Exemple de solution (cactus_sol.txt) :
```
...XX...
...XX.XX
...XX.XX
XX.XX.XX
XX.XX.XX
\...
```

<p align="center">
  <img src="img/cactus_sol.png" alt="cactus solution" width="400"/>
</p>

### Représentation des Données
- **Grille** : Représentée sous forme d’une matrice de valeurs booléennes (`1` pour une case remplie, `0` pour une case vide).
- **Contraintes des lignes et colonnes** : Listes de nombres indiquant les blocs remplis requis.

### Modélisation en SAT
Chaque cellule `(i, j)` de la grille est représentée comme une variable booléenne. Les contraintes sont exprimées sous forme de formules logiques :
1. **Contraintes de ligne** garantissent le bon nombre et regroupement des cellules remplies.
2. **Contraintes de colonne** assurent la même chose pour les colonnes.
3. **Clauses binaires** imposent des espaces entre les blocs lorsque nécessaire.

Le solveur SAT **Gophersat** est utilisé pour résoudre ces contraintes.

### Implémentation
- **Analyse du format d’entrée**
- **Génération des contraintes SAT**
- **Résolution avec un solveur SAT**
- **Affichage visuel de la solution** avec Tkinter

---

In [39]:
%run picross2d/picross2d.py --input_filename lambda.txt

.XX.......
X.XX......
X..X......
...XX.....
....X.....
...XXX....
...XXX....
..XX.XX...
..XX..X...
.XX...XX.X
.XX....XXX
XX.....XX.


## Partie 2 : Solveur de Nonogrammes 3D

### Différences avec le 2D
- Au lieu de lignes et colonnes, nous avons maintenant des **plans 3D**.
- **Trois ensembles de contraintes** (lignes, colonnes, profondeur).
- Modification des indices :
  - **Nombre simple** : Un bloc contigu.
  - **Nombre entouré d’un cercle** : Exactement deux blocs distincts.
  - **Nombre entouré d’un carré** : Trois blocs ou plus distincts.




### Étude de l'explosion combinatoire en 3D

Les solveurs SAT modernes sont très performants, mais leur efficacité est limitée par plusieurs facteurs :

- **Nombre de clauses** (taille de la formule CNF)  
- **Nombre de variables** (chaque cellule du nonogramme 3D devient une variable binaire)  
- **Structure du problème** (certaines formules sont plus difficiles à résoudre que d’autres)  
- **Mémoire nécessaire** (les solveurs SAT utilisent des tableaux et structures pour stocker les clauses)  

---

#### Ordre de grandeur du nombre de clauses en SAT pour un nonogramme

Prenons un **nonogramme 3D** de taille \(N \times N \times N\). Si on encode le problème en une seule passe avec un solveur SAT, on doit représenter :

- **Chaque cellule comme une variable binaire** : \(N^3\) variables  
- **Contraintes de somme sur chaque ligne/colonne/profondeur** : \(O(N^2)\) contraintes par dimension  
- **Clauses de continuité et séparation** : \(O(N^3)\) clauses supplémentaires  

Cela donne un **nombre total de clauses** de l’ordre de **\(O(N^3)\) à \(O(N^4)\)** selon l’encodage.

| **Taille** | **Variables (\(N^3\))** | **Clauses (\(O(N^4)\))** |
|-----------|-----------------|-----------------|
| \(N=5\)  | 125             | ~3,000          |
| \(N=10\) | 1,000           | ~40,000         |
| \(N=15\) | 3,375           | ~150,000        |
| \(N=20\) | 8,000           | ~500,000        |
| \(N=30\) | 27,000          | ~2,500,000      |

Exemple du calcul du nombre de clauses pour **\(N=5\)**
Dès **\(N=20\)**, on a des **centaines de milliers de clauses**, ce qui dépasse déjà les capacités d’un solveur SAT classique en **temps raisonnable**.

---

#### Capacités des principaux solveurs SAT

Les performances des solveurs SAT dépendent du **nombre de clauses** qu’ils peuvent traiter en **temps raisonnable** (quelques minutes à quelques heures). A titre d'exemple voici un aperçu des performances des principaux solveurs SAT actuels:

| **Solveur**       | **Capacité max (ordre de grandeur)** | **Référence** |
|------------------|-----------------------------------|--------------|
| **MiniSat**      | ~\(10^5 - 10^6\) clauses         | (historique, peu optimisé) |
| **Glucose**      | ~\(10^6\) clauses                | (optimisé pour instances difficiles) |
| **CryptoMiniSat** | ~\(10^7\) clauses               | (résout certains SAT industriels) |
| **CaDiCaL**      | ~\(10^7 - 10^8\) clauses         | (très optimisé) |
| **Kissat**       | ~\(10^7 - 10^8\) clauses         | (dernier état de l’art) |
| **Gophersat**    | ~\(10^6 - 10^7\) clauses         | (utilisé en Python, très rapide) |
| **PySAT (interface)** | dépend du backend utilisé   | (supporte plusieurs solveurs) |

---

Ainsi, même avec les meilleurs solveurs SAT, **la résolution complète d’un nonogramme 3D en une seule passe est trop complexe**, ce qui justifie une **approche ligne par ligne** selon le plan suivant:

En raison de l’explosion combinatoire, résoudre toute la grille 3D d’un coup est trop complexe. Une approche plus efficace consiste à :
1. Résoudre chaque **ligne indépendamment**.
2. Utiliser les déductions pour réduire l’espace de recherche.
3. Affiner itérativement jusqu’à obtenir une solution cohérente.


### Plan d’Implémentation
- **Structure des données** : Étendre la représentation 2D en 3D (en utilisant des tableaux NumPy ou des dictionnaires).
- **Contraintes** : Traduire les règles 3D en formules compatibles avec SAT.
- **Approche de résolution** : Implémenter un solveur SAT ligne par ligne.
- **Visualisation** : Utiliser Open3D ou Matplotlib pour l’affichage.

### Références aux Travaux Existants
Nous nous inspirons de projets existants, notamment :
- [Picross 3D Solver](https://github.com/epmjohnston/Picross-3D-Solver)
- Article de recherche : *Generating and solving 3D nonogram puzzles* (Connor Halford, 2016)

---

## Conclusion et Améliorations Possibles
Si le temps le permet, des améliorations possibles incluent :
- **Génération de puzzles** : Créer des nonogrammes solvables à partir de modèles voxelisés.
- **Solveurs optimisés** : Approches hybrides combinant SAT et heuristiques.
- **Interface utilisateur** : Expérience de résolution interactive.